In [1]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)



import os
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re

import seaborn as sns
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model


pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 1)


/Applications/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
Using TensorFlow backend.
/Applications/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Applications/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Applications/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1)

In [2]:
train = pd.read_csv("train.csv",encoding='utf-8')

In [3]:
train.head(20)

,Unnamed: 0,ID,Yoruba,English
0,0,ID_AAJEQLCz,A ṣètò Ìgbìmọ̀ Tó Ń Ṣètò Ìrànwọ́ Nígbà Àjálù láti wá bí àwọn ará wa níbẹ̀ ṣe máa rí àwọn nǹkan tí wọ́n máa nílò lọ́jọ́ iwájú.,A Disaster Relief Committee was formed to organize the long-term relief efforts.
1,1,ID_AASNedba,"Ìrọ̀lẹ́ May 22, 2018 ni wọ́n fàṣẹ ọba mú Arákùnrin Solovyev ní ibùdókọ̀ ojú irin nígbà tí òun àti Anna ìyàwó rẹ̀ ń bọ̀ láti ìrìn-àjò tí wọ́n lọ lórílẹ̀-èdè míì.","Brother Solovyev was arrested on the evening of May 22, 2018, at a railway station, as he was arriving home from a trip abroad with his wife, Anna."
2,2,ID_AAeQrhMq,Iléeṣẹ́ Creative Commons náà,Creative Commons the Organization
3,3,ID_AAxlMgPP,"Pè̩lú Egypt, Morocco àti Tunisia tí wó̩n ti lo̩lé, Senegal nìkan lorílè̩èdè adúláwò̩ kan tó kù, wo̩n á fe̩sè̩wo̩nsè̩ pè̩lú Colombia ni ìfe̩sè̩wo̩nsè̩ tó ké̩yìn ikò̩ H ló̩jó̩bò̩.","With Egypt, Morocco and Tunisia out of the World Cup, Senegal, the only African nation left, will take on Columbia in their last group H game on Thursday."
4,4,ID_ABKuMKSx,"Adájọ́ àgbà lórílẹ̀ èdè Náíjíríà (Attorney General of the Federation), Justice Abubakar Malami ti pàsẹ fún ilé-isẹ́ ọ̀tẹlẹ̀múyẹ́ (Department of State Services, DSS) láti jáwọ́ lórí ẹ̀sun Omoyẹle Sowore ní kíákíá.","The Attorney General of the Federation, Justice Abubakar Malami has ordered the Department of State Services (DSS) to hands off the trial of Omoyele Sowore."
5,5,ID_ABOgDsPv,Ìpilẹ̀ ọrọ̀ọ́ lẹ́gbin.,The beginning of wealth is chock-full of filth.
6,6,ID_ABPBNrvn,"Ní tòótọ́, nígbà tí mo bá bẹ̀rẹ̀ sí sáré káàkiri tí mò ń sọ pé, “Hey, ọmọkùnrin àti ọmọbìnrin, onímọ̀ nípa ààrun ọkàn niyín ní orílẹ̀-ède US, ǹjẹ́ ẹ lè tẹ̀lé mi padà sílé ká lọ ṣe iṣẹ́ pátàkì kan?","In fact, when I start running around saying, ""Hey, boys and girls, you're cardiologists in the U.S, can you come home with me and do a mission?"
7,7,ID_ABRvCeJZ,"""Ní September 2, 2019, Ilé Ẹjọ́ Àgbègbè Zheleznodorozhniy ní ìlú Khabarovsk lórílẹ̀-èdè Rọ́ṣíà dájọ́ pé kí Arákùnrin Valeriy Moskalenko lọ ṣe iṣẹ́ àṣesìnlú fún ọdún méjì àti oṣù méjì.","""On September 2, 2019, the Zheleznodorozhniy District Court of Khabarovsk, Russia, sentenced Brother Valeriy Moskalenko to two years and two months of community service and another six months of probation."
8,8,ID_ABfNHLLl,Olóògùn ní ńṣe bí a-láigbọ́-mọ̀ràn; bí ogun ó bàá wọ̀lú ọlọgbọ́n là ńfọ̀rọ̀ lọ̀.,The medicine man behaves like a person impervious to wise counsel; if war threatens a town the person to consult for counsel is the sage.
9,9,ID_ABifvERa,"Wọ́n mú Anton Lemeshev tó jẹ́ alàgbà, wọ́n sì fi sí àtìmọ́lé fún oṣù méjì kí wọ́n tó gbọ́ ẹjọ́ rẹ̀.","Anton Lemeshev, an elder, was arrested and then sentenced to pretrial detention for two months."


In [4]:
# Check for null values
train.isnull().sum()

Unnamed: 0    0
ID            0
Yoruba        0
English       0
dtype: int64

In [5]:
# Check shape of train dataset
train.shape

(10054, 4)

In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10054 entries, 0 to 10053
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  10054 non-null  int64 
 1   ID          10054 non-null  object
 2   Yoruba      10054 non-null  object
 3   English     10054 non-null  object
dtypes: int64(1), object(3)
memory usage: 314.3+ KB


In [7]:
#lower all cases
train["Yoruba"] = train["Yoruba"].apply(lambda x:x.lower())
train["English"] = train["English"].apply(lambda x:x.lower())

In [8]:
#Removing quotes
train["Yoruba"] = train["Yoruba"].str.replace('"', '')
train["English"] = train["English"].str.replace('"', '')

In [9]:
exclude = set(string.punctuation) # Set of all special characters
# Remove all the special characters
train["Yoruba"]=train["Yoruba"].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
train["English"]=train["English"].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

In [10]:
#Removing numbers from text
remove_digits = str.maketrans("", "", digits)
train["Yoruba"] = train["Yoruba"].apply(lambda x: x.translate(remove_digits))
train["English"] = train["English"].apply(lambda x: x.translate(remove_digits))

# No symbols for 1,2,3 in Yoruba language

In [11]:
# Removing extra spaces
train["Yoruba"] = train["Yoruba"].apply(lambda x: x.strip())
train["English"] = train["English"].apply(lambda x: x.strip())
train["Yoruba"] = train["Yoruba"].apply(lambda x: re.sub(" +", " ", x))
train["English"] = train["English"].apply(lambda x: re.sub(" +", " ", x))

In [12]:
# Add start and end tokens to target sentences
train["Yoruba"] = train["Yoruba"].apply(lambda x: "START_ "+ x +" _END")

In [13]:
train.head()

,Unnamed: 0,ID,Yoruba,English
0,0,ID_AAJEQLCz,START_ a ṣètò ìgbìmọ̀ tó ń ṣètò ìrànwọ́ nígbà àjálù láti wá bí àwọn ará wa níbẹ̀ ṣe máa rí àwọn nǹkan tí wọ́n máa nílò lọ́jọ́ iwájú _END,a disaster relief committee was formed to organize the longterm relief efforts
1,1,ID_AASNedba,START_ ìrọ̀lẹ́ may ni wọ́n fàṣẹ ọba mú arákùnrin solovyev ní ibùdókọ̀ ojú irin nígbà tí òun àti anna ìyàwó rẹ̀ ń bọ̀ láti ìrìnàjò tí wọ́n lọ lórílẹ̀èdè míì _END,brother solovyev was arrested on the evening of may at a railway station as he was arriving home from a trip abroad with his wife anna
2,2,ID_AAeQrhMq,START_ iléeṣẹ́ creative commons náà _END,creative commons the organization
3,3,ID_AAxlMgPP,START_ pè̩lú egypt morocco àti tunisia tí wó̩n ti lo̩lé senegal nìkan lorílè̩èdè adúláwò̩ kan tó kù wo̩n á fe̩sè̩wo̩nsè̩ pè̩lú colombia ni ìfe̩sè̩wo̩nsè̩ tó ké̩yìn ikò̩ h ló̩jó̩bò̩ _END,with egypt morocco and tunisia out of the world cup senegal the only african nation left will take on columbia in their last group h game on thursday
4,4,ID_ABKuMKSx,START_ adájọ́ àgbà lórílẹ̀ èdè náíjíríà attorney general of the federation justice abubakar malami ti pàsẹ fún iléisẹ́ ọ̀tẹlẹ̀múyẹ́ department of state services dss láti jáwọ́ lórí ẹ̀sun omoyẹle sowore ní kíákíá _END,the attorney general of the federation justice abubakar malami has ordered the department of state services dss to hands off the trial of omoyele sowore


In [14]:
# Get English and Yoruba Vocabulary
all_english_words = set()
for english in train["English"]:
    for word in english.split():
        if word not in all_english_words:
            all_english_words.add(word)
            
all_yoruba_words = set()
for yoruba in train["Yoruba"]:
    for word in yoruba.split():
        if word not in all_yoruba_words:
            all_yoruba_words.add(word)

In [15]:
print("Length of all english words: " + str(len(all_english_words)))
print("Length of all yoruba words: " + str(len(all_yoruba_words)))    

Length of all english words: 16317
Length of all yoruba words: 15792


In [16]:
train["length_yor_sentence"] = train["Yoruba"].apply(lambda x:len(x.split(" ")))
train["length_eng_sentence"] = train["English"].apply(lambda x:len(x.split(" ")))

In [17]:
train.head()

,Unnamed: 0,ID,Yoruba,English,length_yor_sentence,length_eng_sentence
0,0,ID_AAJEQLCz,START_ a ṣètò ìgbìmọ̀ tó ń ṣètò ìrànwọ́ nígbà àjálù láti wá bí àwọn ará wa níbẹ̀ ṣe máa rí àwọn nǹkan tí wọ́n máa nílò lọ́jọ́ iwájú _END,a disaster relief committee was formed to organize the longterm relief efforts,29,12
1,1,ID_AASNedba,START_ ìrọ̀lẹ́ may ni wọ́n fàṣẹ ọba mú arákùnrin solovyev ní ibùdókọ̀ ojú irin nígbà tí òun àti anna ìyàwó rẹ̀ ń bọ̀ láti ìrìnàjò tí wọ́n lọ lórílẹ̀èdè míì _END,brother solovyev was arrested on the evening of may at a railway station as he was arriving home from a trip abroad with his wife anna,31,26
2,2,ID_AAeQrhMq,START_ iléeṣẹ́ creative commons náà _END,creative commons the organization,6,4
3,3,ID_AAxlMgPP,START_ pè̩lú egypt morocco àti tunisia tí wó̩n ti lo̩lé senegal nìkan lorílè̩èdè adúláwò̩ kan tó kù wo̩n á fe̩sè̩wo̩nsè̩ pè̩lú colombia ni ìfe̩sè̩wo̩nsè̩ tó ké̩yìn ikò̩ h ló̩jó̩bò̩ _END,with egypt morocco and tunisia out of the world cup senegal the only african nation left will take on columbia in their last group h game on thursday,30,28
4,4,ID_ABKuMKSx,START_ adájọ́ àgbà lórílẹ̀ èdè náíjíríà attorney general of the federation justice abubakar malami ti pàsẹ fún iléisẹ́ ọ̀tẹlẹ̀múyẹ́ department of state services dss láti jáwọ́ lórí ẹ̀sun omoyẹle sowore ní kíákíá _END,the attorney general of the federation justice abubakar malami has ordered the department of state services dss to hands off the trial of omoyele sowore,33,25


In [18]:
train.shape

(10054, 6)

In [19]:
train[train['length_eng_sentence']>220].shape

(0, 6)

In [20]:
train = train[train['length_eng_sentence']<=78]
train = train[train['length_yor_sentence']<=78]

In [21]:
train.shape

(10009, 6)

In [22]:
print("Maximum length of Yoruba Sentence ",max(train['length_yor_sentence']))
print("maximum length of English Sentence ",max(train['length_eng_sentence']))

Maximum length of Yoruba Sentence  78
maximum length of English Sentence  78


In [23]:
max_length_src=max(train['length_yor_sentence'])
max_length_tar=max(train['length_eng_sentence'])

In [24]:
input_words = sorted(list(all_yoruba_words))
target_words = sorted(list(all_english_words))
num_encoder_tokens = len(all_english_words)
num_decoder_tokens = len(all_yoruba_words)
num_encoder_tokens, num_decoder_tokens

(16317, 15792)

In [25]:
num_decoder_tokens += 1 #for zero padding

In [26]:
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])

In [27]:
# Index of the characters
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

In [28]:
train = shuffle(train)
train.head(20)

,Unnamed: 0,ID,Yoruba,English,length_yor_sentence,length_eng_sentence
2166,2166,ID_KxNnJNWI,START_ ọ̀rẹ́ àti alábàáṣe _END,my contacts and associations,5,4
7414,7414,ID_mNvAeonA,START_ ṣé mi ò lẹ́tọ̀ọ́ láti jàǹfààní ohun tí abala kejìdínlọ́gbọ̀n nínú òfin orílẹ̀èdè rọ́ṣíà sọ ni _END,have i lost the right to avail myself of article of the russian constitution,18,14
6347,6347,ID_gofzlFzU,START_ a kì í bú ọba onígẹ̀gẹ̀ lójú àwọn èèyànán ẹ̀ _END,one does not insult a king with a goitre in the presence of his people,12,15
1228,1228,ID_FzecXUlL,START_ àìtó ìpolongo fún àwọn olùgbé ìlú _END,low levels of public awareness,8,5
6797,6797,ID_jGgzaRFh,START_ léyìn èyí ìjọba lè mú àdínkù bá owó iná àti owó omi ní àwọn ilé ìwé nítorí pé bí ó ṣe gbérí yìí ń pa iṣẹ́ wọn lára _END,next the government could lower water and electricity tariffs for schools as these huge costs are hurting their operations,30,19
2696,2696,ID_NlaPdENU,START_ bákan náà àwọn ará lọ́kùnrin lóbìnrin tó gbọ́ èdè àwọn adití lédè korea dáadáa lo àwọn àtẹ tó ní móhùnmáwòrán lára láti máa fi àwọn fídíò èdè adití han ọ̀pọ̀ àwọn adití tó wá síbi ìdíje fáwọn aláàbọ̀ ara _END,additionally brothers and sisters fluent in korean sign language used carts featuring signlanguage videos to welcome the many deaf persons who came to the paralympics,41,25
5604,5604,ID_csybxERe,START_ ìgbà tí a bá rẹni lòwúrọ̀ ẹni _END,whenever one first sees a person that is that persons morning,9,11
4516,4516,ID_XLrjzzVe,START_ kí ló dé tí o ò wọ aṣọ iṣẹ́ _END,why are you not in your uniform,11,7
547,547,ID_CkCKcLXX,START_ ǹjẹ́ aládùúgbò ní ìṣẹ̀lẹ̀àtẹ̀yìnwáa ìfọ́lé _END,does my neighborhood have a history of burglaries,7,8
1576,1576,ID_HtNfqXGN,START_ ṣe ni ẹ̀rí ọkàn àwọn ẹlòmíì ń dá wọn lẹ́bi torí ohun burúkú tí wọ́n ṣe _END,others were tortured by their own consciences because of the terrible deeds they had committed,18,15


In [29]:
#Split the train data into train and test
X, y = train["Yoruba"], train["English"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,random_state=42)
X_train.shape, X_test.shape

((8007,), (2002,))

In [30]:
#Save data
X_train.to_pickle('X_train.pkl')
X_test.to_pickle('X_test.pkl')